In [6]:
# Importar librerías
import pandas as pd
from scipy.stats import t, norm
import numpy as np
import logging

In [5]:
class Muestreador:
    def __init__(self, tamaño_poblacion, varianza: np.array, costos: np.array=np.zeros):
        self.poblacion = tamaño_poblacion
        self.varianza = varianza
        self.costos = costos
    # Métodos para cálculo de tamaño de muestra

    # SRS
    def tamaño_srs(self, error, confianza) -> int:
        z = norm.ppf(1- ((1 - confianza) / 2))
        n = (self.varianza * z**2) / error**2
        return n / (1 + n / self.poblacion)

    # Cálculo de tamaño de muestra por estrato
    def tamaño_estratificado(self, error, confianza, index_kish, proporciones: list) -> dict:
        # Las proporciones se pasan en una lista en el siguiente orden: 
        # Proporción óptima, constante, uniforme
        try:
            tamaños = {"Óptimo": 0, "Kish": 0, "Proporcional": 0, "Uniforme": 0}
            # optima
            numerador = proporciones[0] * np.sqrt(self.varianza) / np.sqrt(self.costos)
            denominador = np.sum(numerador)
            tamaños["Óptimo"] = self.tamaño_srs(error, confianza) * numerador/denominador
            
        # Kish
            numerador = np.sqrt(1 / np.len(proporciones[0])**2 + index_kish * proporciones[0]**(2))
            denominador = np.sum(numerador)
            tamaños["Kish"] = self.tamaño_srs(error, confianza) * numerador / denominador

            # proporcional 
            N_h = self.poblacion * proporciones[1]
            k = self.tamaño_srs(error, confianza) / self.poblacion
            tamaños["Proporcional"] = k * N_h

            # uniforme
            tamaños["Uniforme"] = self.tamaño_srs(error, confianza) / np.len(proporciones[2]) * np.ones_like(proporciones[2]) 

            return tamaños
        except Exception as e:
            logging.error(f"Error al calcular el tamaño de las muestras: {e}")

    # Cálculo de estimadores usando SRS. Se recibe por parámetro una serie de pandas para la cual se calcularán los estadísticos
    def estimadores_SRS(self, variable: pd.Series) -> dict:
        # Leer datos usando parámetro variable para indicar el nombre de la variable aleatoria para los cálculos
        try:
            tamaño_muestra = len(variable) - 1
            fcp = 1 - ( tamaño_muestra / self.poblacion)
            # Cálculo de estimadores
            varianza_poblacional = variable.var()
            total_poblacional = self.poblacion * variable.mean()
            media_poblacional = total_poblacional / self.poblacion
            # Cálculo de errores estándar
            e_total_poblacional = self.poblacion * np.sqrt( fcp* varianza_poblacional / tamaño_muestra)
            e_media_poblacional = np.sqrt(fcp * varianza_poblacional / tamaño_muestra)

            estimadores_y_errores = {"Estimadores" : [total_poblacional, media_poblacional], "Errores": [e_total_poblacional, e_media_poblacional]}
            return estimadores_y_errores
        
        except Exception as e:
            logging.error(f"Error al calcular estimadores con SRS: {e}")

    # Método para calcular estimadores usando muestreo estratificado. Se asume que se recibe una lista con los estratos. 
    # Faltaría adaptar esto a que se reciba un df con los datos por estrato y se itere en él 
    def estimadores_estratificado(self, estratos):
        total_poblacional = 0 
        for estrato in estratos:
            total_poblacional += self.poblacion * 